In [373]:
# =========================================================
# 1. 라이브러리 임포트
# =========================================================
import os, time, re
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm

from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from collections import defaultdict
from pathlib import Path

from typing import List, Dict, Tuple
import string
from difflib import SequenceMatcher

In [374]:
# =========================
# 2. path 설정
# =========================
BASE = Path(os.getcwd())

CLEANED_PATH = BASE / "separated_perfume_cleaned_eng.csv"
ACCORDS_PATH = BASE / "main_accords_all.csv"
OUT_PATH = BASE / "merged_perfume.csv"

print(f"기준 경로: {BASE}")

기준 경로: c:\Workspaces\SKN14-Final-2Team\YooYonghwan


In [375]:
# ------------------------------------------------------
# 3. data load
# 1-1) separated_perfume_cleaned_eng.csv
# ------------------------------------------------------
print("CLEANED_PATH =", CLEANED_PATH)
# cleaned = pd.read_csv(CLEANED_PATH, encoding="utf-8-sig")
cleaned = pd.read_csv(CLEANED_PATH, encoding="cp949")
print(cleaned.info())
# cleaned

CLEANED_PATH = c:\Workspaces\SKN14-Final-2Team\YooYonghwan\separated_perfume_cleaned_eng.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1242 entries, 0 to 1241
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   brand          1242 non-null   object
 1   name           1242 non-null   object
 2   eng_name       1242 non-null   object
 3   size_ml        1242 non-null   object
 4   price_krw      1242 non-null   int64 
 5   detail_url     1242 non-null   object
 6   description    1240 non-null   object
 7   concentration  1230 non-null   object
 8   main_accords   1242 non-null   object
 9   top_notes      1060 non-null   object
 10  middle_notes   1058 non-null   object
 11  base_notes     1060 non-null   object
 12  description_1  1242 non-null   object
dtypes: int64(1), object(12)
memory usage: 126.3+ KB
None


In [376]:
# ------------------------------------------------------
# 3. data load
# 1-2) main_accords_all.csv
# ------------------------------------------------------
print("ACCORDS_PATH =", ACCORDS_PATH)
accords = pd.read_csv(ACCORDS_PATH, encoding="utf-8-sig")
print(accords.info())
# accords

ACCORDS_PATH = c:\Workspaces\SKN14-Final-2Team\YooYonghwan\main_accords_all.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789 entries, 0 to 788
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   perfume_name     789 non-null    object
 1   notes_score      787 non-null    object
 2   season_score     789 non-null    object
 3   day_night_score  789 non-null    object
dtypes: object(4)
memory usage: 24.8+ KB
None


In [377]:
# ------------------------------------------------------
# 4. 원본 데이터 전처리 함수 정의
# ------------------------------------------------------


# 문자열의 공백을 정규화
def norm_space(s: str) -> str:
    return re.sub(r"\s+", " ", s.strip())


# 문자열에 끝에 붙는 특정 접미사 제거 (tester, set, refill) -> 상품명만 남김
def strip_suffixes(s: str) -> str:
    suffixes = [
        r"travel set", r"gift set", r"discovery set", r"sample set",
        r"tester", r"refill", r"duo", r"trio", r"mini", r"set"
    ]
    x = s
    for suf in suffixes:
        # 단어 경계 + 문자열 끝에서만 제거
        x = re.sub(rf"\b{suf}\b\s*$", "", x, flags=re.IGNORECASE).strip()
        x = norm_space(x)
    return x


# 괄호 (), [], {}와 그 안에 있는 내용을 모두 제거
def strip_parens(s: str) -> str:
    x = re.sub(r"\([^)]*\)", " ", s)
    x = re.sub(r"\[[^\]]*\]", " ", x)
    x = re.sub(r"\{[^}]*\}", " ", x)
    return norm_space(x)

# 문자열에 포함된 하이픈(-)을 포함한 모든 구두점 제거
def remove_punct(s: str) -> str:
    table = str.maketrans("", "", string.punctuation)
    return norm_space(s.translate(table))

# 문자열을 소문자로 변환
def casefold(s: str) -> str:
    return s.casefold()

# 두 문자열간의 유사도 측정 (0~1 사이)
def similar(a: str, b: str) -> float:
    return SequenceMatcher(None, a, b).ratio()

# pandas 데이터 프레임에 새로운 열 추가
def add_rule(df: pd.DataFrame, rule: str) -> pd.DataFrame:
    df = df.copy()
    df.insert(0, "merge_rule", rule)
    return df

# ------------------------------------------------------
# 5. 원본 데이터 전처리 수행
# ------------------------------------------------------


# 필드명
ENG_COL = "eng_name"          # cleaned 기준 키
ACC_COL = "perfume_name"      # accords 기준 키

# fuzzy 임계값
FUZZY_THRESHOLD = 0.90

# 각 데이터 셋에서 "제품명"에 대해 None 여부 체크
if ENG_COL not in cleaned.columns:
    raise KeyError("cleaned에 '{}' 컬럼이 없습니다. 실제 컬럼: {}".format(ENG_COL, list(cleaned.columns)))
if ACC_COL not in accords.columns:
    raise KeyError("accords에 '{}' 컬럼이 없습니다. 실제 컬럼: {}".format(ACC_COL, list(accords.columns)))

# 제품명 컬럼에 대해 양 끝 공백 제거(최소한의 안정)
cleaned[ENG_COL] = cleaned[ENG_COL].astype(str).str.strip()
accords[ACC_COL] = accords[ACC_COL].astype(str).str.strip()

# 기존 순서 보존용 인덱스 (cleaned 데이터프레임의 기존 인덱스를 제거하고, 0부터 시작하는 새로운 정수 인덱스를 할당)
cleaned = cleaned.reset_index(drop=True).copy() 
# 1) reset_index(drop=True) : 원래 인덱스 열을 데이터프레임에서 완전히 삭제(데이터프레임의 행 순서를 안정적으로 재설정하기 위함)
# 2) .copy(): reset_index가 반환한 새로운 데이터프레임의 복사본 생성 (원본 cleaned 변수에 대한 예상치 못한 변경을 방지하기 위해 안전하게 작업할 공간을 확보)
cleaned["_rid"] = range(len(cleaned))
# 3) 복사된 데이터프레임에 _rid 라는 새로운 열을 추가  (0부터 시작하여 데이터프레임의 총 행 수(len(cleaned))만큼의 고유한 정수 값이 순서대로 할당)



# accords 사본(규칙용 파생 컬럼)
acc_df = accords.copy()                                             # acc_df : 원본 accords df의 사본을 생성
acc_df["_acc_space"]   = acc_df[ACC_COL].map(norm_space)            # 문자열 공백을 정규화
acc_df["_acc_case"]    = acc_df["_acc_space"].map(casefold)         # 모든문자를 소문자로변환
acc_df["_acc_parens"]  = acc_df["_acc_space"].map(strip_parens)     # 괄호 (), [], {}와 그 안에 있는 내용을 모두 제거
acc_df["_acc_nopunct"] = acc_df["_acc_parens"].map(remove_punct)    # 문자열에 포함된 하이픈(-)을 포함한 모든 구두점 제거

# cleaned 파생
cln_df = cleaned.copy()                                             # acc_df : 원본 accords df의 사본을 생성
cln_df["_eng_space"]   = cln_df[ENG_COL].map(norm_space)            # 문자열 공백을 정규화
cln_df["_eng_case"]    = cln_df["_eng_space"].map(casefold)         # 모든문자를 소문자로변환
cln_df["_eng_suffix"]  = cln_df["_eng_space"].map(strip_suffixes)   # 문자열에 끝에 붙는 특정 접미사 제거 (tester, set, refill) -> 상품명만 남김
cln_df["_eng_parens"]  = cln_df["_eng_space"].map(strip_parens)     # 괄호 (), [], {}와 그 안에 있는 내용을 모두 제거
cln_df["_eng_nopunct"] = cln_df["_eng_parens"].map(remove_punct)    # 문자열에 포함된 하이픈(-)을 포함한 모든 구두점 제거

In [378]:
# =========================
# 6. 전처리 완료 후, 두 데이터셋을 각각 매핑
#    - 각 단계에서 매칭된 cleaned._rid 를 수집하고,
#      다음 단계에서는 남은(미매칭)들만 대상으로 수행
# =========================
matched_ids = set()
results = []  # 단계별 결과 DataFrame 보관

def stage_left_merge(left_df: pd.DataFrame, right_df: pd.DataFrame,
                     left_key: str, right_key: str, rule_name: str) -> pd.DataFrame:
    """
    남은(cln_df 중 미매칭) 행을 left로 하여, 규칙 키로 inner merge
    - left_df: cln_df[~_rid.isin(matched_ids)]
    - right_df: acc_df
    """
    # left에 원본 cleaned 컬럼과 조인키를 포함시키고,
    # right에는 원본 accords 전체 컬럼을 포함시켜 확장되게 함
    merged = left_df.merge(
        right_df, left_on=left_key, right_on=right_key, how="inner", suffixes=("", "_acc")
    )
    if not merged.empty:
        merged = add_rule(merged, rule_name)
    return merged

# 6-1) exact
left = cln_df[~cln_df["_rid"].isin(matched_ids)]
exact = stage_left_merge(left, acc_df, ENG_COL, ACC_COL, "exact")
results.append(exact)
matched_ids.update(exact["_rid"].unique().tolist())

# 6-2) case_only (대소문자 무시)
left = cln_df[~cln_df["_rid"].isin(matched_ids)]
case_only = stage_left_merge(left, acc_df, "_eng_case", "_acc_case", "case_only")
results.append(case_only)
matched_ids.update(case_only["_rid"].unique().tolist())

# 6-3) whitespace_only (공백 정리)
left = cln_df[~cln_df["_rid"].isin(matched_ids)]
ws_only = stage_left_merge(left, acc_df, "_eng_space", "_acc_space", "whitespace_only")
results.append(ws_only)
matched_ids.update(ws_only["_rid"].unique().tolist())

# 6-4) suffix_removed (세트/테스터/리필 등 접미사 제거 후 비교)
left = cln_df[~cln_df["_rid"].isin(matched_ids)]
suf_removed = stage_left_merge(left, acc_df, "_eng_suffix", "_acc_space", "suffix_removed")
results.append(suf_removed)
matched_ids.update(suf_removed["_rid"].unique().tolist())

# 6-5) paren_removed (괄호 제거)
left = cln_df[~cln_df["_rid"].isin(matched_ids)]
par_removed = stage_left_merge(left, acc_df, "_eng_parens", "_acc_parens", "paren_removed")
results.append(par_removed)
matched_ids.update(par_removed["_rid"].unique().tolist())

# 6-6) punct_removed (구두점 제거)
left = cln_df[~cln_df["_rid"].isin(matched_ids)]
pun_removed = stage_left_merge(left, acc_df, "_eng_nopunct", "_acc_nopunct", "punct_removed")
results.append(pun_removed)
matched_ids.update(pun_removed["_rid"].unique().tolist())

# 6-7) fuzzy_suggestion (0.90 이상만)
left = cln_df[~cln_df["_rid"].isin(matched_ids)].copy()
fuzzy_matched = pd.DataFrame()
if not left.empty:
    # accords 후보 사전 (nopunct/casefold 기준으로 비교)
    acc_candidates = acc_df[["_acc_nopunct", ACC_COL]].drop_duplicates().values.tolist()
    # 각 left 행에 대해 best match 하나만 선택
    chosen = []
    for _, r in left.iterrows():
        ln = r["_eng_nopunct"]
        best_sc, best_nm = 0.0, None
        for an_norm, a_name in acc_candidates:
            sc = similar(ln, an_norm)
            if sc > best_sc:
                best_sc, best_nm = sc, a_name
        if best_nm is not None and best_sc >= FUZZY_THRESHOLD:
            chosen.append((r["_rid"], best_nm, best_sc))

    if chosen:
        map_df = pd.DataFrame(chosen, columns=["_rid", "_fuzzy_target_name", "_fuzzy_score"])
        # left(미매칭)와 매핑을 합치고, accords에 target_name으로 조인
        tmp = left.merge(map_df, on="_rid", how="inner")
        fuzzy_matched = tmp.merge(acc_df, left_on="_fuzzy_target_name", right_on=ACC_COL, how="inner")
        if not fuzzy_matched.empty:
            fuzzy_matched = add_rule(fuzzy_matched, "fuzzy_suggestion>=0.90")
            results.append(fuzzy_matched)
            matched_ids.update(fuzzy_matched["_rid"].unique().tolist())

# 6-8) no_suggestion: 끝까지 못 맞춘 애들 그냥 남김(붙이지 않음)
left = cln_df[~cln_df["_rid"].isin(matched_ids)].copy()
if not left.empty:
    # accords 컬럼을 NaN으로 채워서 모양 맞추기
    for col in accords.columns:
        if col not in left.columns:
            left[col] = pd.NA
    left = add_rule(left, "no_suggestion")
    results.append(left)

In [379]:
# =========================
# 7. 결과 결합 및 정렬
# =========================
final = pd.concat(results, ignore_index=True, sort=False)

# 원래 cleaned 컬럼 + accords 컬럼 순서로 정리
cleaned_cols = list(cleaned.columns)  # _rid 포함
accords_cols = [c for c in accords.columns if c not in cleaned_cols]
ordered_cols = ["merge_rule"] + cleaned_cols + accords_cols
final = final[ordered_cols]

# 원래 행 순서 기준으로 정렬(확장된 행은 같은 _rid끼리 묶임)
final = final.sort_values(by=["_rid", "merge_rule"]).reset_index(drop=True)

# helper 컬럼 정리(원한다면 남겨도 됨)
drop_helpers = [c for c in final.columns if c.startswith("_eng_") or c.startswith("_acc_") or c in ["_fuzzy_target_name", "_fuzzy_score"]]
final = final.drop(columns=[c for c in drop_helpers if c in final.columns], errors="ignore")


In [380]:
# =========================
# 8. 저장 및 로그
# =========================
final.to_csv(OUT_PATH, index=False, encoding="utf-8-sig")
print("done. output file:", OUT_PATH.name)

# 간단 요약
summary = final.groupby("merge_rule")["_rid"].nunique().reset_index(name="rows")
print("merge summary by rule:")
print(summary.to_string(index=False))

final


done. output file: merged_perfume.csv
merge summary by rule:
            merge_rule  rows
             case_only    50
                 exact  1003
fuzzy_suggestion>=0.90    98
         no_suggestion    89
         paren_removed     1
        suffix_removed     1


,merge_rule,brand,name,eng_name,size_ml,price_krw,detail_url,description,concentration,main_accords,top_notes,middle_notes,base_notes,description_1,_rid,perfume_name,notes_score,season_score,day_night_score
0,exact,크리드,어벤투스 오 드 퍼퓸,Aventus,50ml,255000,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...,오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향",0,Aventus,fruity(100.0) / sweet(68.4) / woody(66.0) / le...,winter(11.2) / spring(11.3) / summer(3.5) / fa...,day(97.5) / night(99.3)
1,exact,크리드,어벤투스 오 드 퍼퓸,Aventus,100ml,399220,https://www.bysuco.com/product/show/9370,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더...,오 드 퍼퓸,"프루티, 스위트, 레더","베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼","파인애플, 패출리, 모로칸 자스민","자작나무, 머스크, 오크 모스, 암브록산, 시더우드","용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향",1,Aventus,fruity(100.0) / sweet(68.4) / woody(66.0) / le...,winter(11.2) / spring(11.3) / summer(3.5) / fa...,day(97.5) / night(99.3)
2,exact,톰 포드,오드 우드 오 드 퍼퓸,Eau Wood,30ml,179000,https://www.bysuco.com/product/show/10716,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스...,오 드 퍼퓸,"우디, 오우드, 웜 스파이시",NaN,NaN,NaN,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향,2,Eau Wood,warm spicy(100.0) / woody(92.4) / vanilla(76.1...,winter(14.0) / spring(6.5) / summer(31.8) / fa...,day(70.5) / night(30.1)
3,exact,톰 포드,오드 우드 오 드 퍼퓸,Eau Wood,50ml,249000,https://www.bysuco.com/product/show/10716,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 / 오우드 / 웜 스...,오 드 퍼퓸,"우디, 오우드, 웜 스파이시",NaN,NaN,NaN,청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향,3,Eau Wood,warm spicy(100.0) / woody(92.4) / vanilla(76.1...,winter(14.0) / spring(6.5) / summer(31.8) / fa...,day(70.5) / night(30.1)
4,exact,이솝,테싯 오 드 퍼퓸,Tecit,50ml,135000,https://www.bysuco.com/product/show/9970,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 시트러스 / 아로마틱 / ...,오 드 퍼퓸,"시트러스, 아로마틱, 프레쉬 스파이시","유자, 시트러스",바질,"베티버, 클로브",이솝의 시그니처 향기로 따뜻하고 생기넘치며 마음을 릴렉싱 시켜주는 향,4,Tecit,citrus(100.0) / aromatic(93.7) / fresh spicy(9...,winter(28.1) / spring(10.5) / summer(3.1) / fa...,day(93.4) / night(100.0)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1394,exact,에스티 로더,알리아지 스포츠 오 드 퍼퓸,Aliage Sport,50ml,105000,https://www.bysuco.com/product/show/96925,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 얼씨 / 모시 / 우디\n...,오 드 퍼퓸,"얼씨, 모시, 우디","그린노트, 시트러스, 자스민","아르테미지아, 로즈, 너트맥","오크모스, 베티버, 시더",캐주얼한 최초의 스포츠 향수,1237,Aliage Sport,woody(100.0) / earthy(82.9) / mossy(73.4) / ar...,winter(2.3) / spring(19.3) / summer(0.0) / fal...,day(74.6) / night(61.0)
1395,exact,디올,쟈도르 롤러 펄 인피니심 오 드 퍼퓸,J'adore Roller Pearl Infinissime,20ml,89000,https://www.bysuco.com/product/show/225110,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 화이트 플로랄 / 튜베로즈...,오 드 퍼퓸,"화이트 플로랄, 튜베로즈, 우디","블러드 오렌지, 핑크 페퍼, 베르가못","튜베로즈, 일랑일랑, 자스민 삼박, 릴리오브더밸리, 로즈",샌달우드,풍성한 화이트 플로랄의 향기에 우디가 더해진 육감적인 향,1238,J'adore Roller Pearl Infinissime,white floral(100.0) / yellow floral(60.1) / ro...,winter(27.0) / spring(12.4) / summer(7.9) / fa...,day(96.2) / night(73.8)
1396,no_suggestion,조르지오 아르마니,씨 오 드 퍼퓸 리필,C Refill,100ml,130000,https://www.bysuco.com/product/show/693934,[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 우디 /바닐라 / 아로마틱...,오 드 퍼퓸,"우디, 바닐라, 아로마틱",블랙커런트,"메이 로즈, 프리지아","바닐라, 패츌리, 우디 노트, 암브록산",세련되고 관능적인 현대 시프르 향수,1239,NaN,NaN,NaN,NaN
1397,exact,디올,디올 에센스 오 드 뚜왈렛,Dior Essence,100ml,186820,https://www.bysuco.com/product/show/225083,[부향률] \n- 오 드 뚜왈렛\n\n[메인 어코드]\n- 웜 스파이시 / 패츌리 ...,오 드 뚜왈렛,"웜 스파이시, 패츌리, 프레쉬 스파이시",제라늄,시나몬,패츌리,신비로운 관능미를 담은 여성스러운 오리엔탈 향수,1240,Dior Essence,warm spicy(100.0) / woody(88.1) / earthy(87.3)...,winter(3.9) / spring(18.8) / summer(0.8) / fal...,day(55.6) / night(28.7)


In [381]:
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------
# 9. 최종 output 데이터 데이터
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------
#  원본 데이터 (원본 컬럼) 
#   {description} : {[부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더\n\n[메인 노트]\n- 탑 노트: 베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼\n- ....}
#  전처리 후 : 
#   {부향률} : {오 드 퍼퓸}
#   {메인 어코드 : {프루티, 스위트, 레더}		파인애플, 패출리, 모로칸 자스민	자작나무, 머스크, 오크 모스, 암브록산, 시더우드	용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향}
#   {탑 노트} : {베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼}
#   {미들 노트} : {파인애플, 패출리, 모로칸 자스민}
# 	{베이스 노트} : {자작나무, 머스크, 오크 모스, 암브록산, 시더우드}
# 	{향 설명} : {용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향}
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------
#
# 9-1. parse_description(text) : description 컬럼을 부분 추출하여 별도의 컬럼 추가 생성
#   
#   1) _normalize_newlines(text) : description 문자열의 줄바꿈을 정규화(\n)
#   
#   2) [부향률] 
#       2-1) _extract_section(text, r"\[부향률\]") : 부향률 섹션 추출
#       2-2) _cleanup_bullets_and_whitespace() : 불릿(-, *), 공백을 제거 후 깔끔한 형태의 텍스트로 추출 & 컬럼 값 => ex) 부향율 : 오 드 퍼퓸
#
#   3) [메인 어코드]
#       3-1) _extract_section(text, r"\[메인\s*어코드\]") : 메인 어코드 섹션 추출
#       3-2) _cleanup_bullets_and_whitespace() : 불릿(-, *), 공백을 제거 후 깔끔한 형태의 텍스트로 추출 & 컬럼 값 -> ex) 부향율 : 오 드 퍼퓸
#       3-3) _normalize_list_string() : 구분자 변환 ( / -> , ) => ex) '프루티 / 스위트 / 레더' -> '프루티, 스위트, 레더' 형태로 통일.
#
#   4) [메인 노트]
#       4-1) _extract_section(text, r"\[메인\s*노트\]") : 메인 노트 섹션 추출 + 탑/미들/베이스 노트 섹션으로 나누기
#           4-1-1) _extract_note_line(notes_block, "탑 노트")
#           4-1-2) _extract_note_line(notes_block, "미들 노트")
#           4-1-3) _extract_note_line(notes_block, "베이스 노트")
#
#   5) [향 설명] 
#       5-1) _extract_section(text, r"\[향\s*설명\]", take_to_end=True) : 향 설명 섹션 추출
#       5-2) _cleanup_bullets_and_whitespace() : 불릿(-, *), 공백을 제거 후 깔끔한 형태의 텍스트로 추출 & 컬럼 값
#   
#   6) "description" 컬럼 -> 최종 out 딕셔너리 반환 ("부향률", "메인어코드", "메인노트", "향 설명")
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------


# ---------------------------------------------------------------------
# 유틸
# ---------------------------------------------------------------------
def _normalize_newlines(s: str) -> str:
    # \r\n, \r -> \n 통일
    # 리터럴 "\n" 도 실제 개행으로 변환
    return s.replace("\r\n", "\n").replace("\r", "\n").replace("\\n", "\n")

def _cleanup_bullets_and_whitespace(s: str) -> str:
    """
    각 줄 맨앞의 '-', '*' 같은 불릿 제거 후 줄을 공백으로 합쳐 한 줄로 만듭니다.
    """
    lines = []
    for line in s.split("\n"):
        line = re.sub(r"^\s*[-*]\s*", "", line)  # 앞의 불릿 제거
        if line.strip():
            lines.append(line.strip())
    one_line = " ".join(lines)
    one_line = re.sub(r"\s+", " ", one_line).strip()
    return one_line

def _normalize_list_string(s: str) -> str:
    """
    '프루티 / 스위트 / 레더' 같은 문자열을
    '프루티, 스위트, 레더' 형태로 통일.
    """
    # 슬래시, 쉼표(전각 포함) 등으로 분리
    parts = [p.strip() for p in re.split(r"[\/,、\u3001]+", s) if p.strip()]
    return ", ".join(parts)

def _extract_section(text: str, pattern: str, take_to_end=False) -> str:
    """
    [헤더]로 시작하는 섹션을 캡처합니다.
    - take_to_end=True 이면 문서 끝까지, False면 다음 [헤더] 직전까지.
    pattern: 예) r"\[부향률\]"
    """
    if take_to_end:
        m = re.search(pattern + r"\s*(.+)\Z", text, flags=re.S)
    else:
        m = re.search(pattern + r"\s*(.+?)(?=\n\[[^\]]+\]|\Z)", text, flags=re.S)
    return m.group(1).strip() if m else ""

def _extract_note_line(block: str, label_kr: str) -> str:
    """
    메인 노트 블록에서 '- 탑 노트: ...' 처럼 특정 라벨 줄만 뽑아 정리.
    반환은 'A, B, C' 형태 문자열.
    """
    # 줄 단위 탐색 (불릿/공백 허용, 콜론 형태 허용)
    mm = re.search(
        rf"(?m)^\s*-\s*{label_kr}\s*노트\s*[:：]\s*(.+?)\s*$",
        block
    )
    if not mm:
        return ""
    raw = mm.group(1).strip()
    return _normalize_list_string(raw)

# ---------------------------------------------------------------------
# 메인 파서
# ---------------------------------------------------------------------
def parse_description(text):
    if not isinstance(text, str):
        text = ""
    text = _normalize_newlines(text)

    out = {
        "부향률": "",
        "메인 어코드": "",
        "탑 노트": "",
        "미들 노트": "",
        "베이스 노트": "",
        "향 설명": "",
    }

    # [부향률] : 다음 [헤더] 전까지
    sec = _extract_section(text, r"\[부향률\]")
    if sec:
        out["부향률"] = _cleanup_bullets_and_whitespace(sec)

    # [메인 어코드] : 다음 [헤더] 전까지 -> 리스트 정규화 후 "A, B, C"
    sec = _extract_section(text, r"\[메인\s*어코드\]")
    if sec:
        sec = _cleanup_bullets_and_whitespace(sec)
        out["메인 어코드"] = _normalize_list_string(sec)

    # [메인 노트] 블록
    notes_block = _extract_section(text, r"\[메인\s*노트\]")
    if notes_block:
        out["탑 노트"]    = _extract_note_line(notes_block, "탑")
        out["미들 노트"]  = _extract_note_line(notes_block, "미들")
        out["베이스 노트"] = _extract_note_line(notes_block, "베이스")

    # [향 설명] : 문서 끝까지 포함 (하단 * 안내문까지 포함)
    sec = _extract_section(text, r"\[향\s*설명\]", take_to_end=True)
    if sec:
        out["향 설명"] = _cleanup_bullets_and_whitespace(sec)

    return out

# ---------------------------------------------------------------------
# 실행
# ---------------------------------------------------------------------
parsed = final["description"].apply(parse_description) # "description" 컬럼의 각 행에 parse_description() 함수 적용 -> 딕셔너리 반환
parsed_df = pd.DataFrame(parsed.tolist())           # (딕셔너리) 파싱결과를 리스트로 변환

# 원본과 합치고 저장 (Excel 호환을 위해 utf-8-sig 권장)
final_df = pd.concat([final, parsed_df], axis=1)
final_df.to_csv("merged_perfume.csv", index=False, encoding="utf-8-sig")

print(final_df[["부향률", "메인 어코드", "탑 노트", "미들 노트", "베이스 노트", "향 설명"]].head(3))




      부향률           메인 어코드                         탑 노트               미들 노트  \
0  오 드 퍼퓸     프루티, 스위트, 레더  베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼  파인애플, 패출리, 모로칸 자스민   
1  오 드 퍼퓸     프루티, 스위트, 레더  베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼  파인애플, 패출리, 모로칸 자스민   
2  오 드 퍼퓸  우디, 오우드, 웜 스파이시                                                    

                         베이스 노트  \
0  자작나무, 머스크, 오크 모스, 암브록산, 시더우드   
1  자작나무, 머스크, 오크 모스, 암브록산, 시더우드   
2                                 

                                                향 설명  
0  용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 일부 상품에 한하여 신규로 리...  
1  용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 일부 상품에 한하여 신규로 리...  
2             청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향  


In [382]:
# ---------------------------------------------------------------------
# 10. 공지 문구 제거
# ---------------------------------------------------------------------

# NOTICE_TEXT = "일부 상품에 한하여 신규로 리뉴얼 된 디자인의 제품이 발송될 수 있습니다. 이로 인한 교환/반품은 어려운 점 참고 부탁드립니다."
NOTICE_TEXT = "본 제품은 아시아에서 유통되는 상품이 아닌, 유럽에서 유통되는 상품입니다."

def clean_notice(desc: str) -> str:
    """향 설명에서 공지 문구 제거"""
    if not isinstance(desc, str):
        return desc
    # 지정된 공지 문구 제거
    cleaned = desc.replace(NOTICE_TEXT, "")
    # 불필요한 공백 정리
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned

# 적용
final_df["향 설명"] = final_df["향 설명"].apply(clean_notice)

# 저장
final_df.to_csv("merged_perfume.csv", index=False, encoding="utf-8-sig")

print(final_df["향 설명"].head(10))

0    용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 일부 상품에 한하여 신규로 리...
1    용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 일부 상품에 한하여 신규로 리...
2               청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향
3               청량한 소나무 계열의 향과 부드러운 침구가 부드럽게 감싸주는 듯한 향
4               이솝의 시그니처 향기로 따뜻하고 생기넘치며 마음을 릴렉싱 시켜주는 향
5    감귤에서 느낄 수 있는 시트러스 계열과 우디 계열의 자연스러운 조합으로 산속 깨끗한...
6    감귤에서 느낄 수 있는 시트러스 계열과 우디 계열의 자연스러운 조합으로 산속 깨끗한...
7                               이국적인 매력과 자연스러움이 돋보이는 향
8                              감각의 물 또는 에센스의 물로 해석되는 향
9                              감각의 물 또는 에센스의 물로 해석되는 향
Name: 향 설명, dtype: object


In [383]:
# -----------------------------
# 11. 환경 변수 로드
# -----------------------------
load_dotenv(override=True)                 # .env 파일에서 환경 변수 로드

def getenv_clean(k: str) -> str:           # 환경 변수에서 값 가져오기
    v = os.getenv(k, "") or ""             # 기본값은 빈 문자열
    return v.strip().strip('"').strip("'") # 양쪽 따옴표 제거

OPENAI_API_KEY         = getenv_clean("OPENAI_API_KEY")
PINECONE_API_KEY       = getenv_clean("PINECONE_API_KEY")
OPENAI_EMBEDDING_MODEL = getenv_clean("OPENAI_EMBEDDING_MODEL") or "text-embedding-3-small"

INDEX_NAME = "perfume-search-score-0825"                   # Pinecone 인덱스명
CLOUD, REGION = "aws", "us-east-1"
EMBED_DIM  = 1536  # text-embedding-3-small

print("[DEBUG] OPENAI:", (OPENAI_API_KEY[:6]+"...") if OPENAI_API_KEY else "(empty)")
print("[DEBUG] PINECONE:", (PINECONE_API_KEY[:6]+"...") if PINECONE_API_KEY else "(empty)")

[DEBUG] OPENAI: sk-pro...
[DEBUG] PINECONE: pcsk_6...


In [384]:
# -----------------------------
# 12. 인덱스 생성
# -----------------------------
pc = Pinecone(api_key=PINECONE_API_KEY) # Pinecone 클라이언트 초기화

def ensure_index(name=INDEX_NAME, dim=EMBED_DIM, cloud=CLOUD, region=REGION): 
    
    names = [x["name"] for x in pc.list_indexes()]                            # 현재 존재하는 인덱스 목록 가져오기
    
    if name not in names:                                                     # 인덱스가 존재하지 않으면 생성
        print(f"[INFO] Creating Pinecone index '{name}' ...")                 
        pc.create_index(                                                      # 인덱스 생성
            name=name,
            dimension=dim,
            metric="cosine",
            spec=ServerlessSpec(cloud=cloud, region=region)                   # Specify cloud and region (aws, us-east-1)
        )

        while not pc.describe_index(name).status.get("ready"): # 인덱스가 준비될 때까지 대기
            time.sleep(0.5)

    idx = pc.Index(name)                # 인덱스 객체 가져오기
    _ = idx.describe_index_stats()      # 인덱스 상태 확인

    print(f"[OK] Index ready: {name}") # 인덱스 준비 완료 메시지 출력

    return idx                         # 인덱스 객체 반환

index = ensure_index()                 # 인덱스 생성함수 호출

[OK] Index ready: perfume-search-score-0825


In [385]:
# -----------------------------
# 12. notes_score, season_score, day_night_score 문자열 파싱 및 정규화
# -----------------------------
def parse_scores(score_string: str, normalize: bool = False) -> str:
    display_str_parts: List[str] = []
    
    if not isinstance(score_string, str) or score_string.strip() == "blank":
        return ""

    try:
        matches = re.findall(r'(\w+)\((\d+\.?\d*)\)', score_string)
        scores = {note_name: float(score_val) for note_name, score_val in matches}
        
        if normalize:
            total = sum(scores.values())
            if total > 0:
                normalized_scores = {k: v / total for k, v in scores.items()}
                display_str_parts = [f"{k}:{v * 100:.0f}%" for k, v in normalized_scores.items()]
            else:
                display_str_parts = [f"{k}:0%" for k in scores.keys()]
        else:
            display_str_parts = [f"{k}:{v:.0f}%" for k, v in scores.items()]
    except (ValueError, AttributeError):
        return ""
    
    return ", ".join(display_str_parts) if display_str_parts else ""

# -----------------------------
# 14. CSV → Document 변환 
# -----------------------------

def csv_to_documents_selected_columns(csv_path: str) -> List[Document]:
    df = pd.read_csv(csv_path).fillna("blank")
    docs: List[Document] = []
    
    # 1. 본문에 포함할 컬럼
    page_content_cols = [
        col for col in df.columns.tolist() 
        if col not in ["notes_score", "season_score", "day_night_score"]
    ]

    # 2. metadata로 사용할 컬럼
    metadata_cols = ["name", "brand", "price_krw", "size_ml", "main_accords"]

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Build Documents", unit="row"):
        # 점수 필드 파싱
        notes_display_str = parse_scores(row.get('notes_score', 'blank'), normalize=False)
        season_display_str = parse_scores(row.get('season_score', 'blank'), normalize=True)
        day_night_display_str = parse_scores(row.get('day_night_score', 'blank'), normalize=True)

        # page_content 구성
        parts = [f"{col}: {str(row.get(col, 'blank'))}" for col in page_content_cols if str(row.get(col, 'blank')).strip() != "blank"]
        page_content = " | ".join(parts) if parts else " "

        # metadata 구성
        metadata = {col: str(row.get(col, 'blank')) for col in metadata_cols}
        if notes_display_str:
            metadata['notes_score_display'] = f"{{ {notes_display_str} }}"
        if season_display_str:
            metadata['season_score_display'] = f"{{ {season_display_str} }}"
        if day_night_display_str:
            metadata['day_night_score_display'] = f"{{ {day_night_display_str} }}"

        docs.append(Document(page_content=page_content, metadata=metadata))

    print(f"[INFO] Built {len(docs)} Documents.")
    return docs


# -----------------------------
# 15. CSV → Document 변환 
# -----------------------------
try:
    csv_path = 'merged_perfume.csv'
    docs = csv_to_documents_selected_columns(csv_path)

    if docs:
        for i in range(min(3, len(docs))):
            print(f"\n--- Document {i+1} ---")
            print("Page Content:", docs[i].page_content)
            print("Metadata:", docs[i].metadata)
except FileNotFoundError:
    print("CSV 파일을 찾을 수 없습니다. 경로를 확인해주세요.")


Build Documents: 100%|██████████| 1399/1399 [00:00<00:00, 2311.72row/s]

[INFO] Built 1399 Documents.

--- Document 1 ---
Page Content: merge_rule: exact | brand: 크리드 | name: 어벤투스 오 드 퍼퓸 | eng_name: Aventus | size_ml: 50ml | price_krw: 255000 | detail_url: https://www.bysuco.com/product/show/9370 | description: [부향률] \n- 오 드 퍼퓸\n\n[메인 어코드]\n- 프루티 / 스위트 / 레더\n\n[메인 노트]\n- 탑 노트: 베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼\n- 미들 노트: 파인애플, 패출리, 모로칸 자스민\n- 베이스 노트: 자작나무, 머스크, 오크 모스, 암브록산, 시더우드\n\n[향 설명]\n- 용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향\n\n*일부 상품에 한하여 신규로 리뉴얼 된 디자인의 제품이 발송될 수 있습니다.\n이로 인한 교환/반품은 어려운 점 참고 부탁드립니다. | concentration: 오 드 퍼퓸 | main_accords: 프루티, 스위트, 레더 | top_notes: 베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼 | middle_notes: 파인애플, 패출리, 모로칸 자스민 | base_notes: 자작나무, 머스크, 오크 모스, 암브록산, 시더우드 | description_1: 용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 | _rid: 0 | perfume_name: Aventus | 부향률: 오 드 퍼퓸 | 메인 어코드: 프루티, 스위트, 레더 | 탑 노트: 베르가못, 블랙 커런트, 애플, 레몬, 핑크 페퍼 | 미들 노트: 파인애플, 패출리, 모로칸 자스민 | 베이스 노트: 자작나무, 머스크, 오크 모스, 암브록산, 시더우드 | 향 설명: 용기와 힘, 비전, 그리고 성공을 기원하는 고급진 향 일부 상품에 한하여 신규로 리뉴얼 된 디자인의 제품이 발송될 수 있습니다. 이로 

In [386]:
# -----------------------------
# 16. Pinecone upsert 함수
# -----------------------------

# LangChain 문서 객체를 사용하기 위해 임포트합니다.
from langchain_core.documents import Document

# Pinecone 인덱스에 문서를 추가하는 데 사용할 임베딩 모델과 벡터 스토어를 정의합니다.
embeddings = OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL, api_key=OPENAI_API_KEY)
vector_store = PineconeVectorStore(index_name=INDEX_NAME, embedding=embeddings)

def upsert_documents(docs: list[Document], batch_size: int = 100, id_prefix="perfume"):
    """
    주어진 문서 리스트를 Pinecone 벡터 스토어에 업로드(upsert)합니다.
    """
    n = len(docs)
    if n == 0:
        print("[WARN] no docs")
        return
    print(f"Upserting {n} docs (batch={batch_size}) ...")

    for i in tqdm(range(0, n, batch_size), desc="Upsert", unit="batch"):
        # 현재 배치의 문서들을 선택합니다.
        batch = docs[i:i + batch_size]
        # 각 문서에 대한 고유 ID를 생성합니다.
        ids = [f"{id_prefix}-{i + j}" for j in range(len(batch))]

        try:
            # 배치 단위로 문서를 벡터 스토어에 업로드합니다.
            vector_store.add_documents(batch, ids=ids)
        except Exception as e:
            print(f"[ERROR] 업서트 실패 (docs {i} ~ {i + len(batch) - 1}): {e}")

    # 업로드 완료 후 인덱스 상태를 확인합니다.
    stats = pc.Index(INDEX_NAME).describe_index_stats()
    print("Done. total_vector_count:", stats.get("total_vector_count"))
    return stats



In [387]:
# -----------------------------
# 16. Pinecone upsert 수행
# -----------------------------
# [필요시 1회] CSV → 문서(all columns) → Pinecone 업서트
docs = csv_to_documents_selected_columns(OUT_PATH)
stats = upsert_documents(docs, batch_size=100)

Build Documents: 100%|██████████| 1399/1399 [00:00<00:00, 1824.32row/s]


[INFO] Built 1399 Documents.
Upserting 1399 docs (batch=100) ...


Upsert: 100%|██████████| 14/14 [01:30<00:00,  6.46s/batch]


Done. total_vector_count: 1399


In [388]:
# -----------------------------
# 17. Vector DB 조회 함수
# -----------------------------

# 출력 컬럼 정의
STANDARD_COLS = ["brand", "name", "향 설명", "메인 어코드", "탑 노트", "미들 노트", "베이스 노트"]

# 실제 컬럼명 -> df display 컬럼명 매핑
KEY_MAPPING = {
    "brand": ["brand"],
    "name": ["name"],
    "향 설명": ["description_1", "향 설명", "description"],
    "메인 어코드": ["main_accords", "메인 어코드"],
    "탑 노트": ["top_notes", "탑 노트"],
    "미들 노트": ["middle_notes", "미들 노트"],
    "베이스 노트": ["base_notes", "베이스 노트"]
}

# Vector DB 조회 함수
def search(field_queries: Dict[str, Tuple[str, float]], top_k: int = 5) -> pd.DataFrame:
    scores = defaultdict(float)
    docs_by_id = {}

    for field, (query, weight) in field_queries.items():
        full_query = f"{field}: {query}"
        try:
            docs = vector_store.as_retriever(search_kwargs={"k": top_k}).invoke(full_query)
        except Exception as e:
            print(f"❌ '{field}' 검색 오류: {e}")
            continue

        for doc in docs:
            doc_id = doc.metadata.get("detail_url", doc.page_content[:50])
            scores[doc_id] += weight
            if doc_id not in docs_by_id:
                docs_by_id[doc_id] = doc

    if not docs_by_id:
        print("⚠️ 검색 결과 없음.")
        return pd.DataFrame()

    sorted_docs = sorted(docs_by_id.items(), key=lambda x: -scores[x[0]])
    top_docs = [doc for _, doc in sorted_docs[:top_k]]

    rows = []
    for d in top_docs:
        meta = d.metadata or {}

        # page_content 파싱: 'key: value | key: value' 형태 분리
        parsed_from_content = {
            k.strip(): v.strip()
            for item in d.page_content.split(" | ") if ":" in item
            for k, v in [item.split(":", 1)]
        }

        row_vals = []
        for col in STANDARD_COLS:
            val = ""
            for mapped_key in KEY_MAPPING.get(col, [col]):
                val = (
                    meta.get(mapped_key)
                    or parsed_from_content.get(mapped_key)
                    or ""
                )
                if val:
                    break
            val = val.replace("\r\n", "\\n").replace("\n", "\\n")
            row_vals.append(val)

        row_dict = dict(zip(STANDARD_COLS, row_vals))
        rows.append(row_dict)

    return pd.DataFrame(rows, columns=STANDARD_COLS)


In [389]:
# -----------------------------
# 18. Vector DB 조회
# -----------------------------
field_queries = {
    # "향 설명": ("남자다운", 10.0),
    "메인 어코드": ("시트러스", 10.0),
    "탑 노트": ("베르가못", 1.0)
}

df_results = search(field_queries, top_k=10)
display(df_results)


,brand,name,향 설명,메인 어코드,탑 노트,미들 노트,베이스 노트
0,아쿠아 디 파르마,콜로니아 퓨투라 오 드 코롱,시트러스와 우디함이 어우러진 생동감 있고 꾸밈 없는 이미지를 연출,"시트러스, 아로마틱, 프레쉬 스파이시","베르가못, 레몬, 자몽, 핑크 페퍼","라벤더, 세이지",베티버
1,아쿠아 디 파르마,콜로니아 인텐자 오 드 코롱,콜로니아에 우디와 레더가 가미되어 가장 남성적인 매력을 연출,"시트러스, 웜 스파이시, 아로마틱","칼라브리안 베르가못, 진저, 카다멈, 시칠리안 레몬","네롤리, 아르테미지아, 머틀","레더, 시더, 벤조인, 머스크, 패츌리"
2,에르메스,오 드 시트롱 느와르 오 드 코롱,많은 사랑을 받는 감귤류 과일인 레몬의 역설을 드러내는 스모키하면서도 신선함을 표현한 향,"시트러스, 스모키, 우디",,,
3,산타 마리아 노벨라,루사 오 드 코롱,스파이시한 꽃과 평온한 네롤리의 부드러운 조화,"시트러스, 아로마틱, 프레쉬 스파이시","베르가못, 네롤리, 아말피 레몬, 비터 오렌지, 로즈","페티그레인, 라벤더, 레몬 버베나, 클로브, 로즈마리","머스크, 캐스토리움, 발삼, 시벳, 벤조인"
4,이니시오 퍼퓸,미스틱 익스페리언스 오 드 퍼퓸,머스크와 커피의 강렬하고 품격 있는 조합과 초월적인 경험을 추구하는 입문자를 위한 향수,"우디, 앰버, 파우더리",,,
5,산타 마리아 노벨라,시타 디 교토 오 드 코롱,일본의 연꽃과 피렌체의 아이리스의 조화,"우디, 파우더리, 아이리스","산사나무, 히야신스, 베르가못, 일랑일랑, 오렌지, 자스민, 로즈","아이리스, 사이프러스, 자작나무, 로투스, 라벤더, 플럼, 피치, 시나몬","인센스, 가이악우드, 에보니우드, 오리스, 버지니아 시더, 샌달우드, 머스크, 앰버..."
6,산타 마리아 노벨라,에바 오 드 코롱,상큼한 레몬향과 스파이시하고 드라이한 우드향이 어우러진 향,"시트러스, 우디, 아로마틱","베르가못, 레몬","페퍼, 스파이시 노트","베티버, 우디 노트"
7,메종 프란시스 커정,아쿠아 셀레스티아 오 드 뚜왈렛,파란 하늘과 푸른 바다의 맑고 평온한 상쾌함을 담은 향,"그린, 시트러스, 아로마틱","라임, 민트, 블랙 커런트, 네롤리","그린 노트, 미모사, 로즈",화이트 머스크
8,아쿠아 디 파르마,베르가모또 디 칼라브리아 오 드 뚜왈렛,베르가못과 진저 머스크의 조합으로 청쾌한 휴식을 즐기는 이미지 연출,"시트러스, 아로마틱","베르가못, 시트론","진저, 시더, 플라워 노트","베티버, 머스크, 벤조인"
9,산타 마리아 노벨라,멜로그라노 오 드 코롱,달달한 플로럴 오리엔탈 노트와 은은한 비누 잔향을 전합니다,"우디, 얼시, 앰버","베르가못, 스파이시, 비터 오렌지","석류, 일랑일랑, 로즈","오크모스, 랍다넘, 페출리, 머스크"
